# Data Compilation

In [1]:
# System modules
import os
import exdir as ex
import warnings
warnings.filterwarnings('always', category=UserWarning)
warnings.filterwarnings('always', category=DeprecationWarning)

# Data processing modules
import numpy as np
import nsync
from scipy.ndimage import gaussian_filter1d

# Visualization modules
import plotly.graph_objects as go

### Compile Data per FOV

In [2]:
root = "./data"
output_path = os.path.join("./", "compiled_data_2")
if not os.path.exists(output_path):
    os.mkdir(output_path)

f = ex.File(os.path.join(output_path, "data.exdir"), mode="w", allow_remove=True)

if os.path.isdir(root):
    for day in sorted(os.listdir(root)):
        day_path = os.path.join(root, day)
        if not os.path.isdir(day_path):
            continue

        # print(f"Processing: {day}")

        day_group_name = day.replace(" ", "_")
        # day_group = f.create_group(day_group_name)
        day_group_dataset = []
        day_num_neurons = 0

        for animal in sorted(os.listdir(day_path)):
            animal_path = os.path.join(day_path, animal)
            if not os.path.isdir(animal_path):
                continue

            # print(f"  -> {animal}")

            animal_group_name = animal.replace(" ", "_")
            # animal_group = day_group.create_group(animal_group_name)

            for fov in sorted(os.listdir(animal_path)):
                fov_path = os.path.join(animal_path, fov)
                if not os.path.isdir(fov_path):
                    continue

                # print("  " * 2 + f"  -> {fov}")

                fov_group_name = fov.replace(" ", "_")
                # fov_group = animal_group.create_group(fov_group_name)

                extracted_signals_files = sorted([os.path.join(fov_path, f) for f in os.listdir(fov_path) if f.endswith(".npy") and "extractedsignals_raw" in f])
                behavior_event_log_files = sorted([os.path.join(fov_path, f) for f in os.listdir(fov_path) if f.endswith(".mat")])

                dataset = nsync.NSyncDataset(behavior_event_log_files, extracted_signals_files)
                day_num_neurons += dataset.num_neurons
                print(dataset.extract_unique_events())

                # extracted_windows = dataset.extract_normalized_windows()

        print(f"Day: {day}, Num Neurons: {day_num_neurons}")

[7.000000e+00 3.629260e+05 3.629260e+02 1.088778e+04]
[7.000000e+00 8.546890e+05 8.546890e+02 2.564067e+04]
[7.000000e+00 1.882559e+06 1.882559e+03 5.647677e+04]
[7.00000e+00 1.79778e+05 1.79778e+02 5.39334e+03]
[7.00000e+00 2.38315e+05 2.38315e+02 7.14945e+03]
[[7.000000e+00 3.629260e+05 3.629260e+02 1.088778e+04]
 [7.000000e+00 8.546890e+05 8.546890e+02 2.564067e+04]
 [7.000000e+00 1.882559e+06 1.882559e+03 5.647677e+04]
 [7.000000e+00 1.797780e+05 1.797780e+02 5.393340e+03]
 [7.000000e+00 2.383150e+05 2.383150e+02 7.149450e+03]]
[7.00000e+00 8.54440e+04 8.54440e+01 2.56332e+03]
[7.00000e+00 2.07216e+05 2.07216e+02 6.21648e+03]
[7.000000e+00 4.271250e+05 4.271250e+02 1.281375e+04]
[7.000000e+00 7.745020e+05 7.745020e+02 2.323506e+04]
[[7.000000e+00 8.544400e+04 8.544400e+01 2.563320e+03]
 [7.000000e+00 2.072160e+05 2.072160e+02 6.216480e+03]
 [7.000000e+00 4.271250e+05 4.271250e+02 1.281375e+04]
 [7.000000e+00 7.745020e+05 7.745020e+02 2.323506e+04]]
[7.0000e+00 3.0706e+04 3.0706e+01

In [3]:
infile = ex.File("./compiled_data_2/data", mode="r")
# dataset = infile["1_MidAcq"]["dataset"]["event_window_extracted_signals"]

fig = go.Figure()

for day in infile.keys():
    dataset = None
    try:
        if "dataset" in infile[day].keys():
            dataset = infile[day]["dataset"]["event_window_extracted_signals"]
            print(f"Day: {day}, Neurons: {dataset.shape[0]}")
        else:
            continue
    except Exception as e:
        print(f"Error loading {day}: {e}")

    if dataset:
        mean_signal = gaussian_filter1d(np.nanmean(dataset, axis=0), sigma=10)
        # fig.add_trace(
        #     go.Scatter(
        #         y=mean_signal,
        #         mode='lines',
        #         name=f'{day} Mean Signal'
        #     )
        # )

        # median_signal = np.median(dataset, axis=0)
        # fig.add_trace(
        #     go.Scatter(
        #         y=median_signal,
        #         mode='lines',
        #         name='Median Signal'
        #     )
        # )

fig.update_layout(
    title='Signal Over Time',
    xaxis_title='Sample Index',
    yaxis_title='Signal Value',
    legend_title="Dataset",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    width=800,
    height=600
)

fig.show()
